In [13]:
# %%
import tensorflow as tf
from utils.data_preprocess import load_data
from utils.module import model_eval
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from utils.model_evaluation import plot_test_pred
import pandas as pd

# Sandwiched dropout layer to the hyperparameter tuned model and did the predictions 1000 times without training the model again
# Sandwiched dropout layer to the hyperparameter tuned model and did the predictions 1000 times after training the model again
# During hyperparameter tuning, it is ensured that a dropout layer is there after a dense layer. This model is directly used for uncertainity quanitification

In [14]:
single_leakage, two_leakage = load_data()
# two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

# single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
data['x2'] = data['x2'].replace(np.nan, 0)
data['y2'] = data['y2'].replace(np.nan, 0)

data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual', 'total flow rate'
       ])

y = data[['x1', 'y1', 'x2', 'y2', "leak_2"]]
x = data.drop(['x1', 'y1', 'x2', 'y2', "leak_2"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 


# sort x1, y1 and x2, y2. coordinates with lowest x will take the position of x1
def coords_swap(y1):
    s = y1['x2'] < y1['x1']
    y1.loc[s, ['x1','x2']] = y1.loc[s, ['x2','x1']].values
    y1.loc[s, ['y1','y2']] = y1.loc[s, ['y2','y1']].values
    return y1

y1_test_unsc = y_test

y1_data = [y_train, y_val, y_test]
y1_data_types = ['y_train', 'y_val', 'y_test']
for y1_data_types, y1 in zip(y1_data_types, y1_data):
    y1_data_types = coords_swap(y1)

y1_train = y_train[['x1', 'y1']]
y1_test = y_test[['x1', 'y1']]
y1_val = y_val[['x1', 'y1']]

y2_train = y_train[['x2', 'y2']]
y2_test = y_test[['x2', 'y2']]
y2_val = y_val[['x2', 'y2']]


y3_train = y_train[["leak_2"]]
y3_test = y_test[["leak_2"]]
y3_val = y_val[["leak_2"]]

y1_columns = y1_train.columns
y2_columns = y2_train.columns
y3_columns = y3_train.columns
X_columns = X_train.columns

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

y1_train = pd.DataFrame(y1_train, columns=y1_columns)
y1_test = pd.DataFrame(y1_test, columns=y1_columns)
y1_val = pd.DataFrame(y1_val, columns=y1_columns)

scaler_coords2 = StandardScaler()
y2_train = scaler_coords2.fit_transform(y2_train)
y2_test = scaler_coords2.transform(y2_test)
y2_val = scaler_coords2.transform(y2_val)

y2_train = pd.DataFrame(y2_train, columns=y2_columns)
y2_test = pd.DataFrame(y2_test, columns=y2_columns)
y2_val = pd.DataFrame(y2_val, columns=y2_columns)

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

X_train = pd.DataFrame(X_train, columns=X_columns)
X_test = pd.DataFrame(X_test, columns=X_columns)
X_val = pd.DataFrame(X_val, columns=X_columns)

# y_train = [y1_train, y2_train]
# y_val = [y1_val, y2_val]
# y_test = [y1_test, y2_test]

y3_train = y3_train.reset_index().drop(columns='sample_number')
y3_val = y3_val.reset_index().drop(columns='sample_number')
y3_test = y3_test.reset_index().drop(columns='sample_number')

y_train_all = pd.concat([y1_train, y2_train, y3_train], axis=1)
y_test_all = pd.concat([y1_test, y2_test, y3_test], axis=1)
y_val_all = pd.concat([y1_val, y2_val, y3_val], axis=1)

X_train_np, y1_train_np, y2_train_np, y3_train_np = X_train.values, y1_train.values, y2_train.values, y3_train.values
X_val_np, y1_val_np, y2_val_np, y3_val_np = X_val.values, y1_val.values, y2_val.values, y3_val.values
X_test_np, y1_test_np, y2_test_np, y3_test_np = X_test.values, y1_test.values, y2_test.values, y3_test.values

# Create TensorFlow datasets from NumPy arrays.
batch_size = 32
buffer_size = len(X_train)  # Set the buffer size to the number of training examples for full shuffling.

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y1_train_np, y2_train_np , y3_train_np))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_np, y1_val_np, y2_val_np, y3_val_np))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y1_test_np, y2_test_np, y3_test_np))

# Shuffle, batch, and prefetch the training dataset.
train_dataset_org = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Batch the validation and test datasets.
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [15]:
def masked_mse(y_true, y_pred):
    mask = tf.keras.backend.cast(tf.keras.backend.not_equal(y_true, 0), tf.keras.backend.floatx())
    mse = tf.keras.backend.mean(tf.keras.backend.square(y_true - y_pred) * mask)
    return mse

In [18]:
from tensorflow import keras
from keras.utils import CustomObjectScope

with CustomObjectScope({'masked_mse':masked_mse}):
              model = tf.keras.models.load_model('saved_model/Multi_leak/experiment5/Mask_MTL_sameLoss')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 160)          1760        ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 512)          82432       ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 416)          213408      ['dense_1[0][0]']                
                                                                                              

In [20]:
# %%
dropout_prob = 0.1

stoch_model = tf.keras.Sequential()

for i, layer in enumerate(model.layers):
    stoch_model.add(layer)
    # Add your intermediate layer after each existing layer
    if i == 0:
        continue
    if i == len(model.layers)-1:
        continue
    intermediate_layer = tf.keras.layers.Dropout(dropout_prob)
    stoch_model.add(intermediate_layer)

# Compile the new model
stoch_model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=model.optimizer.lr.numpy()),
                loss="mse",
                metrics='mae')
# Print the summary of the new model
# stoch_model.fit(X_train, y_train, epochs=1000, validation_data = (X_val, y_val), shuffle= True)
# stoch_model.summary()
# model_evaluate, y_pred = model_eval(stoch_model, X_test, y_test, X_train, y_train, X_val, y_val)
# # # # %%
# pred=np.stack([stoch_model(X_test,training=True) 
#                for sample in range(1000)])
# predictions_list = pred.tolist()
# predictions_list_unsc = []
# # print(len(predictions_list))
# for pred in predictions_list:
#     pred = scaler_coords.inverse_transform(pred)
#     predictions_list_unsc.append(pred)
# predictions__unsc = np.array(predictions_list_unsc)

# # print(predictions__unsc.shape)
# pred_mean=predictions__unsc.mean(axis=0)
# pred_std = predictions__unsc.std(axis=0) 
# # print(pred_mean.shape, pred_std.shape)
# # # %%
# y_test = scaler_coords.inverse_transform(y_test)
# plot_test_pred(y_test, pred_mean)
# # print(pred_std)


ValueError: Exception encountered when calling layer "dense_8" (type Dense).

Dimensions must be equal, but are 160 and 416 for '{{node dense_8/MatMul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false](dense_8/Cast, dense_8/MatMul/ReadVariableOp)' with input shapes: [?,160], [416,160].

Call arguments received by layer "dense_8" (type Dense):
  • inputs=tf.Tensor(shape=(None, 160), dtype=float32)

In [ ]:
# def Average(lst):
#     return sum(lst)/len(lst)
# print(Average(radius.tolist()))

In [ ]:
import numpy as np

# Generate 100 random 2D arrays for demonstration (replace with your own data)
num_arrays = 1000
array_size = (54, 2)  # Replace with the actual size of your arrays
arrays = predictions_list_unsc

# Initialize a variable to store the sum of L2 distances
sum_of_distances = 0.0

# Calculate the sum of L2 distances between all pairs of arrays
for i in range(num_arrays):
    for j in range(i + 1, num_arrays):  # Avoid calculating the distance for the same pairs twice
        distance = np.linalg.norm(arrays[i] - arrays[j])  # Calculate L2 distance
        sum_of_distances += distance

# Calculate the average L2 distance
radius = sum_of_distances / (num_arrays * (num_arrays - 1) / 2)

print("Average L2 Distance:", radius)

In [ ]:
pred_std.mean(axis=0)

In [ ]:
import matplotlib.pyplot as plt

x_range = np.arange(180, 16048, 250)
y_range = np.arange(100, 5233, 250)
X, Y = np.meshgrid(x_range, y_range)

def plot_test_pred_uncertainity(test, pred, radius):
    plt.figure(figsize=(40, 20))
    
    # plt.title(f'Sample Number {sample_number}', fontsize=20)
    
    # plot sensor positions
    sensors = np.array([[2426, 70], [5480, 70], [8661, 191], [11676, 584], [13976, 917], [2603, 5163], [5723, 5163], [8417, 5103], [11646, 4740], [14641, 4391]])
    for i in range(len(sensors)):
        plt.scatter(sensors[i, 0], sensors[i, 1], color='tab:green', s=300)
        if i < 5:
            plt.text(sensors[i, 0], sensors[i, 1] - 200, 'MFC'+str(i+1), fontsize='xx-large')
        else:
            plt.text(sensors[i, 0], sensors[i, 1] + 350, 'MFC'+str(i+1), fontsize='xx-large')

    # plot leakage positions
    plt.scatter(X, Y, color='black', s=10)
    # radius = radius.tolist()
    
    for i in range(len(test)):
        if i != len(test)-1:
            plt.scatter(test[i][0], test[i][1], color='tab:red', s=200, alpha= 0.7)
            plt.scatter(pred[i][0], pred[i][1], s=radius, color='tab:blue', alpha= 0.7)
            line = np.vstack((test[i], pred[i])).transpose()
            plt.plot(line[0], line[1], color = 'black')
            plt.annotate(str(i), (pred[i][0], pred[i][1] + 0.2), fontsize=15)
            plt.annotate(str(i), (test[i][0], test[i][1] + 0.2), fontsize=15)
        else:
            plt.scatter(test[i][0], test[i][1], color='tab:red', s=200, label="true", alpha= 0.7)
            plt.scatter(pred[i][0], pred[i][1], s=radius, color='tab:blue', label="pred", alpha= 0.7)
            line = np.vstack((test[i], pred[i])).transpose()
            plt.plot(line[0], line[1], color = 'black')
            plt.annotate(str(i), (pred[i][0], pred[i][1] + 0.2), fontsize=15)
            plt.annotate(str(i), (test[i][0], test[i][1] + 0.2), fontsize=15)

    # print(X.shape)
    legend = plt.legend(loc='upper right')
    # legend.legendHandles[0]._legmarker.set_markersize(15)
    # legend.legendHandles[0]._legmarker.set_alpha(1)
    # plot wing contour
    plot_wing_contour()

    # include grid coordinate system
    plt.hlines(0, -1000, 17000, linestyle='dashed')
    plt.hlines(2600, -1000, 17000, linestyle='dashed')
    plt.hlines(5233, -1000, 17000, linestyle='dashed')
    plt.vlines(0, -1000, 6000, linestyle='dashed')
    plt.vlines(7930, -1000, 6000, linestyle='dashed')
    plt.vlines(16048, -1000, 6000, linestyle='dashed')
    plt.text(-850, -75, '$y = 0$', fontsize=20)
    plt.text(-850, 2600-75, '$y = 2600$', fontsize=20)
    plt.text(-850, 5233-75, '$y = 5233$', fontsize=20)
    plt.text(75, -700, '$x=0$', fontsize=20)
    plt.text(7930+75, -700, '$x=7930$', fontsize=20)
    plt.text(16048+75, -700, '$x=16048$', fontsize=20)
    # plt.text(180, 5800, f'(x1, y1) = ({x1}, {y1}) = ({j1-31}, {-i1+10})', fontsize=20)
    plt.legend(loc="upper left")
    # invert y axis
    plt.gca().invert_yaxis()

    lgnd = plt.legend(loc="upper left", numpoints=2, fontsize=15)

    #change the marker size manually for both lines
    lgnd.legendHandles[0]._sizes = [30]
    lgnd.legendHandles[1]._sizes = [30]

    plt.savefig('./results/hypermodel_results.png')

    plt.show()

def plot_wing_contour():
    plt.plot([0, 7930], [0, 0], 'k')
    plt.plot([7930, 16048], [0, 1149], 'k')
    plt.plot([16048, 16048], [1149, 4386], 'k')
    plt.plot([16048, 7843], [4386, 5233], 'k')
    plt.plot([7843, 2493], [5233, 5233], 'k')
    plt.plot([2493, 0], [5233, 0], 'k')
    plt.xlim([-1000, 17000])
    plt.ylim([-1000, 6000])
    plt.xticks([])
    plt.yticks([])
    plt.gca().set_aspect('equal')



plot_test_pred_uncertainity(y_test, pred_mean, radius)

In [ ]:
import matplotlib.pyplot as plt

x_range = np.arange(180, 16048, 250)
y_range = np.arange(100, 5233, 250)
X, Y = np.meshgrid(x_range, y_range)

def plot_test_pred_uncertainity(test, pred, radius, std):
    plt.figure(figsize=(40, 20))
    
    # plt.title(f'Sample Number {sample_number}', fontsize=20)
    
    # plot sensor positions
    sensors = np.array([[2426, 70], [5480, 70], [8661, 191], [11676, 584], [13976, 917], [2603, 5163], [5723, 5163], [8417, 5103], [11646, 4740], [14641, 4391]])
    for i in range(len(sensors)):
        plt.scatter(sensors[i, 0], sensors[i, 1], color='tab:green', s=300)
        if i < 5:
            plt.text(sensors[i, 0], sensors[i, 1] - 200, 'MFC'+str(i+1), fontsize='xx-large')
        else:
            plt.text(sensors[i, 0], sensors[i, 1] + 350, 'MFC'+str(i+1), fontsize='xx-large')

    # plot leakage positions
    plt.scatter(X, Y, color='black', s=10)
    radius = radius.tolist()
    
    for i in range(len(test)):
        if i != len(test)-1:
            plt.scatter(test[i][0], test[i][1], color='tab:red', s=200, alpha= 0.7)
            plt.scatter(pred[i][0], pred[i][1], s=200, color='tab:blue', alpha= 0.7)
            plt.errorbar(test[i][0], test[i][1], xerr=pred_std[i][0], yerr=pred_std[i][1], fmt='o', color='tab:grey', capsize=4)
            line = np.vstack((test[i], pred[i])).transpose()
            plt.plot(line[0], line[1], color = 'black')
            plt.annotate(str(i), (pred[i][0], pred[i][1] + 0.2), fontsize=15)
            plt.annotate(str(i), (test[i][0], test[i][1] + 0.2), fontsize=15)
        else:
            plt.scatter(test[i][0], test[i][1], color='tab:red', s=200, label="True", alpha= 0.7)
            plt.scatter(pred[i][0], pred[i][1], s=200, color='tab:blue', label= "Prediction", alpha= 0.7)
            plt.errorbar(test[i][0], test[i][1], xerr=pred_std[i][0], yerr=pred_std[i][1], fmt='o', color='tab:grey', capsize=4, label='Uncertainty')
            line = np.vstack((test[i], pred[i])).transpose()
            plt.plot(line[0], line[1], color = 'black')
            plt.annotate(str(i), (pred[i][0], pred[i][1] + 0.2), fontsize=15)
            plt.annotate(str(i), (test[i][0], test[i][1] + 0.2), fontsize=15)

    # print(X.shape)
    # legend = plt.legend(loc='upper right')
    # legend.legendHandles[0]._legmarker.set_markersize(15)
    # legend.legendHandles[0]._legmarker.set_alpha(1)
    # plot wing contour
    plot_wing_contour()

    # include grid coordinate system
    plt.hlines(0, -1000, 17000, linestyle='dashed')
    plt.hlines(2600, -1000, 17000, linestyle='dashed')
    plt.hlines(5233, -1000, 17000, linestyle='dashed')
    plt.vlines(0, -1000, 6000, linestyle='dashed')
    plt.vlines(7930, -1000, 6000, linestyle='dashed')
    plt.vlines(16048, -1000, 6000, linestyle='dashed')
    plt.text(-850, -75, '$y = 0$', fontsize=20)
    plt.text(-850, 2600-75, '$y = 2600$', fontsize=20)
    plt.text(-850, 5233-75, '$y = 5233$', fontsize=20)
    plt.text(75, -700, '$x=0$', fontsize=20)
    plt.text(7930+75, -700, '$x=7930$', fontsize=20)
    plt.text(16048+75, -700, '$x=16048$', fontsize=20)
    # plt.text(180, 5800, f'(x1, y1) = ({x1}, {y1}) = ({j1-31}, {-i1+10})', fontsize=20)
    plt.legend(loc="upper left")
    # invert y axis
    plt.gca().invert_yaxis()

    # lgnd = plt.legend(loc="upper left", numpoints=2, fontsize=15)

    #change the marker size manually for both lines
    # lgnd.legendHandles[0]._sizes = [30]
    # lgnd.legendHandles[1]._sizes = [30]

    plt.savefig('./results/hypermodel_results.png')

    plt.show()

def plot_wing_contour():
    plt.plot([0, 7930], [0, 0], 'k')
    plt.plot([7930, 16048], [0, 1149], 'k')
    plt.plot([16048, 16048], [1149, 4386], 'k')
    plt.plot([16048, 7843], [4386, 5233], 'k')
    plt.plot([7843, 2493], [5233, 5233], 'k')
    plt.plot([2493, 0], [5233, 0], 'k')
    plt.xlim([-1000, 17000])
    plt.ylim([-1000, 6000])
    plt.xticks([])
    plt.yticks([])
    plt.gca().set_aspect('equal')


plot_test_pred_uncertainity(y_test, pred_mean, radius, pred_std)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

x_range = np.arange(180, 16048, 250)
y_range = np.arange(100, 5233, 250)
X, Y = np.meshgrid(x_range, y_range)

def plot_test_pred_uncertainity(test, pred, radius, std):
    plt.figure(figsize=(40, 20))
    
    # plt.title(f'Sample Number {sample_number}', fontsize=20)
    
    # plot sensor positions
    sensors = np.array([[2426, 70], [5480, 70], [8661, 191], [11676, 584], [13976, 917], [2603, 5163], [5723, 5163], [8417, 5103], [11646, 4740], [14641, 4391]])
    for i in range(len(sensors)):
        plt.scatter(sensors[i, 0], sensors[i, 1], color='tab:green', s=300)
        if i < 5:
            plt.text(sensors[i, 0], sensors[i, 1] - 200, 'MFC'+str(i+1), fontsize='xx-large')
        else:
            plt.text(sensors[i, 0], sensors[i, 1] + 350, 'MFC'+str(i+1), fontsize='xx-large')

    # plot leakage positions
    plt.scatter(X, Y, color='black', s=10)
    radius = radius.tolist()
    
    for i in range(len(test)):
        if i != len(test)-1:
            plt.scatter(test[i][0], test[i][1], color='tab:red', s=200, alpha= 0.7)
            plt.scatter(pred[i][0], pred[i][1], s=200, color='tab:blue', alpha= 0.7)
            line = np.vstack((test[i], pred[i])).transpose()
            plt.plot(line[0], line[1], color = 'black')
            plt.annotate(str(i), (pred[i][0], pred[i][1] + 0.2), fontsize=15)
            plt.annotate(str(i), (test[i][0], test[i][1] + 0.2), fontsize=15)
        else:
            plt.scatter(test[i][0], test[i][1], color='tab:red', s=200, label="True", alpha= 0.7)
            plt.scatter(pred[i][0], pred[i][1], s=200, color='tab:blue', label= "Prediction", alpha= 0.7)
            line = np.vstack((test[i], pred[i])).transpose()
            plt.plot(line[0], line[1], color = 'black')
            plt.annotate(str(i), (pred[i][0], pred[i][1] + 0.2), fontsize=15)
            plt.annotate(str(i), (test[i][0], test[i][1] + 0.2), fontsize=15)


    for i in range(len(pred)):
        mean_x, mean_y = pred[i][0], pred[i][1]
        sigma_x, sigma_y = std[i][0], std[i][1]

        # Plot ellipses representing uncertainty
        ellipse = patches.Ellipse((mean_x, mean_y), width=2*sigma_x, height=2*sigma_y, angle=0, alpha=0.2)
        plt.gca().add_patch(ellipse)
    plt.gca().set_aspect('equal')

    plot_wing_contour()

    # include grid coordinate system
    plt.hlines(0, -1000, 17000, linestyle='dashed')
    plt.hlines(2600, -1000, 17000, linestyle='dashed')
    plt.hlines(5233, -1000, 17000, linestyle='dashed')
    plt.vlines(0, -1000, 6000, linestyle='dashed')
    plt.vlines(7930, -1000, 6000, linestyle='dashed')
    plt.vlines(16048, -1000, 6000, linestyle='dashed')
    plt.text(-850, -75, '$y = 0$', fontsize=20)
    plt.text(-850, 2600-75, '$y = 2600$', fontsize=20)
    plt.text(-850, 5233-75, '$y = 5233$', fontsize=20)
    plt.text(75, -700, '$x=0$', fontsize=20)
    plt.text(7930+75, -700, '$x=7930$', fontsize=20)
    plt.text(16048+75, -700, '$x=16048$', fontsize=20)
    # plt.text(180, 5800, f'(x1, y1) = ({x1}, {y1}) = ({j1-31}, {-i1+10})', fontsize=20)
    plt.legend(loc="upper left")
    # invert y axis
    plt.gca().invert_yaxis()

    # lgnd = plt.legend(loc="upper left", numpoints=2, fontsize=15)

    #change the marker size manually for both lines
    # lgnd.legendHandles[0]._sizes = [30]
    # lgnd.legendHandles[1]._sizes = [30]

    plt.savefig('./results/hypermodel_results.png')

    plt.show()

def plot_wing_contour():
    plt.plot([0, 7930], [0, 0], 'k')
    plt.plot([7930, 16048], [0, 1149], 'k')
    plt.plot([16048, 16048], [1149, 4386], 'k')
    plt.plot([16048, 7843], [4386, 5233], 'k')
    plt.plot([7843, 2493], [5233, 5233], 'k')
    plt.plot([2493, 0], [5233, 0], 'k')
    plt.xlim([-1000, 17000])
    plt.ylim([-1000, 6000])
    plt.xticks([])
    plt.yticks([])
    plt.gca().set_aspect('equal')

plot_test_pred_uncertainity(y_test, pred_mean, radius, pred_std)